In [1]:
# Read a mat file
import scipy
import os

data_folder = "TSR_1_data"
gt_data_path = os.path.join(data_folder, "ground_truth.mat")
gt_data_mat = scipy.io.loadmat(gt_data_path)
gt_data = gt_data_mat["ground_truth"]

In [2]:
# Separate the the gt_data by timestamps
gt_data_timestamps = []

timestamps = gt_data.shape[2]
for t in range(timestamps):
    gt_data_at_t = gt_data[:, :, t]
    gt_data_timestamps.append(gt_data_at_t)

In [3]:
# Clean unrecognized objects: remove the entries whose axies 1 are all zeros
import numpy as np

for t in range(len(gt_data_timestamps)):
    gt_data_at_t = gt_data_timestamps[t]
    mask = ~(gt_data_at_t == 0).all(axis=1)
    gt_data_at_t = gt_data_at_t[mask]
    # If no traffic sign detected, make the gt_data_at_t empty (reserve negative samples)
    if len(gt_data_at_t) == 0:
        gt_data_at_t = []
    gt_data_timestamps[t] = gt_data_at_t

In [4]:
# # sanity check of convert_prescan_gt_to_yolo_gt()
# from utils_own import *

# # Image dimensions (example)
# image_width = 800
# image_height = 600


# # prescan_gt = np.array([10, 0.748750, 0.553333, 0.085000, 0.470000])
# # yolo_gt = convert_prescan_gt_to_yolo_gt(prescan_gt, image_width, image_height)

# import cv2



# # Normalized bounding box (x, y, w, h)
# # _, x, y, w, h = yolo_gt
# _, x, y, w, h = [10, 0.748750, 0.553333, 0.085000, 0.470000]
# # Conver xywh to pixel values
# x = x * image_width
# y = y * image_height
# w = w * image_width
# h = h * image_height

# # y = image_height - y
# # Convert normalized coordinates to pixel values
# left = int(x  - w / 2)
# top = int(y - h / 2)
# right = int(x + w / 2)
# bottom = int(y + h / 2)

# # left = 0
# # top = 0
# # right = 100
# # bottom = 100
# # print(right, left, top, bottom)

# # Load an image (example)
# # image = cv2.imread(r"D:\PreScan2302_Experiments\PrescanDemoAMC_HJY\Results\SimResults_20250423_153743\CameraSensor_1\CameraSensor_1_00000.png")
# image = cv2.imread(r"D:\HJY\Prescan_data_collecting\CameraSensor_1\00244.png")
# # image = cv2.imread(r"D:\PreScan2302_Experiments\PrescanDemoAMC_HJY\Results\SimResults_20250428_153917\CameraSensor_1\CameraSensor_1_00245.png")


# # Draw the bounding box (color: green, thickness: 2)
# cv2.rectangle(image, (left, bottom), (right, top), (0, 255, 0), 2)

# # Show the image with bounding box
# cv2.imshow("Image with Bounding Box", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [5]:
# Make a dictionary mapping numeric ids (of Prescan) to class ids (yolo format)

# Load from mat file
numid_to_classid_mat = scipy.io.loadmat("numid_to_classid.mat")

numid_to_classid_array = numid_to_classid_mat['numid_to_classid_mat']
numid_to_classid = {}
for i in range(len(numid_to_classid_array)):
    numid = numid_to_classid_array[i, 0]
    classid = numid_to_classid_array[i, 1]
    numid_to_classid[numid] = classid

In [6]:
# Filter out the traffic signs, which will only be used
traffic_numids = list(numid_to_classid.keys())

for t in range(len(gt_data_timestamps)):
    gt_data_at_t = gt_data_timestamps[t]
    mask = np.isin(gt_data_at_t[:, 0], traffic_numids)
    gt_data_at_t=  gt_data_at_t[mask]
    if len(gt_data_at_t) == 0:
        gt_data_at_t = []
    gt_data_timestamps[t] = gt_data_at_t
        

In [ ]:
# Parse the Prescan gt data into YOLO gt format and save into "labels" folder
IMAGE_WIDTH = 800
IMAGE_HEIGHT = 600

from utils_own import *
import os

import shutil

# if os.path.exists("labels"): shutil.rmtree("labels")
os.makedirs("labels", exist_ok=True)
# os.makedirs("labels_gtsdb_format", exist_ok=True)

for t in range(len(gt_data_timestamps)):
    gt_data_at_t = gt_data_timestamps[t]
    with open(f"labels/{t:05}.txt", "a") as file:
        # Separate rows
        for row in gt_data_at_t:
            yolo_gt = convert_prescan_gt_to_yolo_gt(row, IMAGE_WIDTH, IMAGE_HEIGHT, numid_to_classid, gtsdb_format=False)
            file.write(yolo_gt)

In [3]:
# sanity check of generated "labels"
import cv2

image_width = 800
image_height = 600

image = cv2.imread(r"D:\HJY\graduation_perception\TSR_1_data\CameraSensor_1\00026.png")
label_path = r"D:\HJY\graduation_perception\labels\00026.txt"

with open(label_path, "r") as f:
    for row in f:
        _, x, y, w, h = map(float, row.split())
        # Conver xywh to pixel values
        x = x * image_width
        y = y * image_height
        w = w * image_width
        h = h * image_height

        # y = image_height - y
        # Convert xywh to ltrb
        left = int(x  - w / 2)
        top = int(y - h / 2)
        right = int(x + w / 2)
        bottom = int(y + h / 2)
        # bottom = int(top + (bottom - top) * (0.6 / (0.6 + 1.6)))

        print(left, top, right, bottom)
        # Draw the bounding box (color: green, thickness: 2)
        cv2.rectangle(image, (left, bottom), (right, top), (0, 255, 0), 2)

# Show the image with bounding box
cv2.imshow("Image with Bounding Box", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


250 278 264 327
617 278 631 327
469 291 474 308
351 291 356 308
371 292 374 298
381 293 383 297
301 296 302 297
333 296 334 297
347 296 348 297
379 296 380 297
154 296 155 300
173 296 174 297
200 296 201 297


In [1]:
# Change image file names to the same as the label file names
import os

img_folder = "TSR_1_data\CameraSensor_1"
label_folder = "labels"
label_file_names = os.listdir(label_folder)
img_file_names = os.listdir(img_folder)

# Delete the non-image file in the image folder
for file_name in img_file_names:
    if not file_name.endswith(".png"):
        file_to_delete = os.path.join(img_folder, file_name)
        os.remove(file_to_delete)
        break

img_file_names = os.listdir(img_folder)
# Delete the last image if the images have one more extra than the labels
if len(img_file_names) > len(label_file_names):
    img_to_delete = img_file_names[-1]
    img_to_delete_path = os.path.join(img_folder, img_to_delete)
    os.remove(img_to_delete_path)

for ind, img_file_name in enumerate(os.listdir(img_folder)):
    img_path = os.path.join(img_folder, img_file_name)

    # Get the corresponding label file name
    label_file_name = label_file_names[ind]

    # Make the new img file name (replace .txt with .png)
    new_file_name = label_file_name.replace(".txt", ".png")
    new_img_path = os.path.join(img_folder, new_file_name)

    # Rename the image file
    os.rename(img_path, new_img_path)


In [ ]:
# # Validate the yolo model performance on the collected dataset
# from ultralytics import YOLO

# model_path = r"D:\HJY\UL_YOLO\last.pt"
# model = YOLO(model_path)
# metrics = model.val(data="D:\HJY\Prescan_data_collecting\dataset\prescan.yaml")

Ultralytics 8.3.110  Python-3.8.7 torch-2.4.1+cpu CPU (Intel Core(TM) i9-10980XE 3.00GHz)
YOLO11s summary (fused): 100 layers, 9,429,441 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 61.98.4 MB/s, size: 508.5 KB)


val: Scanning D:\HJY\Prescan_data_collecting\dataset\val\labels... 300 images, 128 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 431.13it/s]


val: New cache created: D:\HJY\Prescan_data_collecting\dataset\val\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:17<00:00,  1.11it/s]


                   all        300        492          0          0          0          0
        speed limit 20        106        106          0          0          0          0
        speed limit 30        120        120          0          0          0          0
        speed limit 50        136        136          0          0          0          0
        speed limit 60         31         31          0          0          0          0
        traffic signal         99         99          0          0          0          0
Speed: 0.4ms preprocess, 45.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\val6
